# Repeat 노드

In [1]:
import numpy as np

D, N = 8, 7
x = np.random.rand(1,D) #1행 8열
print(x.shape)


(1, 8)


In [2]:
y = np.repeat(x, N, axis=0) #행(axis=0)으로 N번 x를 반복하라 
print(y.shape)

(7, 8)


In [3]:
dy = np.random.rand(N, D)
print(dy.shape)

(7, 8)


In [5]:
dx = np.sum(dy, axis=0, keepdims=True) #형상은 유지하면서 행끼리만 합하여 줄어들어라
print(dx.shape)

(1, 8)


In [6]:
dx

array([[2.84011822, 3.22488201, 3.11661333, 4.13722795, 3.12249185,
        1.93298259, 3.392369  , 4.48449468]])

# SUM 노드

In [12]:
import numpy as np

N, D = 7, 8
x = np.random.rand(N, D)
print(x.shape) #7행 8열

y = np.sum(x, axis=0, keepdims=True) #sum 노드 순전파
print(y.shape) #1행 8열



(7, 8)
(1, 8)


In [13]:
dy = np.random.rand(1, D) # 날라온 미분값
print(dy.shape)
dx = np.repeat(dy, N, axis=0) #sum 노드 역전파
print(dx.shape)

(1, 8)
(7, 8)


# Matmul

In [2]:
class MatMul:
    # bias addition 까지 있으면 affine
    # 없고 행렬의 곱만 있다면 MatMul
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)] # np.zeroes_like returns an array of zeros with the same shape and type as a given array "W"
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout):
        # y = xW    (1)
        # dout이 뒤에서 날라온 미분
        W, = self.params 
        dx = np.dot(dout, W.T) # x편미분 -> eqn (1) 의 x자리에 dout, 그리고 W자리에 W의 전치 (W.T)
        dW = np.dot(self.x.T, dout) # W편미분 -> eqn (1) 의 W자리에 dout, 그리고 x자리에 x의 전치 (x.T)
        self.grads[0][...] = dW # deep copy
        return dx

In [6]:
import numpy as np
a = np.array([1,2,3])
b = np.array([4,5,6])

a = b # Shallow copy -> reference point of variable a moves to the object that b was pointing at
      # As reference count for np.array([1,2,3]) has reduced to 1, the object gets destroyed 
print(a) # [4 5 6]
print(id(a))
a[0] = 10
print(id(a))
print(id(a) == id(b)) # True
print(a)
print(b)


[4 5 6]
4685704288
4685704288
True
[10  5  6]
[10  5  6]


In [9]:
import numpy as np
a=np.array([1,2,3])
b=np.array([4,5,6])

a[...] = b # Deep copy
print(a)
print(id(a))
print(id(a)==id(b)) # False
a[0] = 10 # only a[0] changes, b[0] stays the same
print(a) # [10 5 6]
print(b) # [4 5 6]

[4 5 6]
4719440240
False
[10  5  6]
[4 5 6]


# 시그모이드 계층

In [10]:
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [],[]
        self.out = None

    def forward(self, x): # 시그모이드 순전파
        out = 1 / (1+np.exp(-x))
        self.out = out # 역전파의 y 으로 쓰기 위해 반드시 저장해놔야 함
        return out

    def backward(self, dout): # 시그모이드 역전파
        dx = dout * (1.0 - self.out) * self.out # dy/dx = y(1-y) where y = dout (뒤에서 날라온 미분값), 미분으로 나온 값을 저장했다가 (y) 그것 곱하기 (1-y) 
        return dx

# Affine 계층

In [11]:
class Affine:
    # Matmul과 유사하나 bias 덧셈까지 있음
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
    
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b # explicit 하지 않지만 +b에는 행 repeat가 숨어있음 
        self.x = x
        return out

    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0) # 위의 bias의 repeat이 있기 때문에 repeat의 미분, 즉 sum 필요

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

# Softmax with Loss 계층

In [12]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None # softmax 출력
        self.t = None # 정답 레이블

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x) # 순전파일 경우 softmax 따로, CEE 따로 

        # 정답 레이블이 원 핫 벡터일 경우 정답의 인덱스로 변환
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1 # 역전파일 경우 그냥 (예측값-실질값). 정답일 경우 1이기 떄문에 정답을 맞춘 것만 골라서 1을 빼줌
                                               # np.arange(batch_size) => 배치가 10개라고 했을 때 np.arange 는 0에서 9행까지 라는 뜻이므로 np.arange(batch_size) 는 미니 배치의 모든 행에 대해서 라는 뜻 
                                               # self.t = 열의 값. dx[np.arange(batch_size), self.t]으로 하면 정답 자리에 있는 예측값만 뽑아옴
        dx *= dout # 예측값-실질값 * 뒤에서 날라온 미분값.
        dx = dx/batch_size # 미니배치로 여러개 날라온다면 배치사이즈로 디비젼 하여 평균값 내기

        return dx


# 가중치 갱신

In [13]:
class SGD:
    '''
    확률적 경사하강법 (Stochastic Gradient Descent)
    '''

    def __init__(self, lr = 0.01):
        self.lr = lr

    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i] # params[] = Weight, lr = learning rate, grads[] = 미분값
